# La Liga matches

In [ ]:
# https://github.com/statsbomb/statsbombpy
!pip install statsbombpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.1 MB/s eta 0:00:00


In [ ]:
import warnings
from statsbombpy.api_client import NoAuthWarning

warnings.simplefilter("ignore", NoAuthWarning)

In [ ]:
from statsbombpy import sb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
competitions=sb.competitions()
# We get the names of the competitions
competitions.competition_name.unique()

array(['1. Bundesliga', 'Champions League', 'Copa del Rey',
       "FA Women's Super League", 'FIFA U20 World Cup', 'FIFA World Cup',
       'Indian Super league', 'La Liga', 'Liga Profesional', 'Ligue 1',
       'North American League', 'NWSL', 'Premier League', 'Serie A',
       'UEFA Euro', 'UEFA Europa League', "UEFA Women's Euro",
       "Women's World Cup"], dtype=object)

In [ ]:
matches_count_per_competition = {}

for i, row in competitions.iterrows():
    competition_id = row['competition_id']
    season_id = row['season_id']
    try:
        matches = sb.matches(competition_id, season_id)
        matches_count = len(matches)
        competition_name = row['competition_name']

        # Number of matches for the same competition across different seasons
        if competition_name in matches_count_per_competition:
            matches_count_per_competition[competition_name] += matches_count
        else:
            matches_count_per_competition[competition_name] = matches_count
    except Exception as e:
        print(f"Error for competition_id: {competition_id}, season_id: {season_id}. Error message: {e}")

for competition, count in matches_count_per_competition.items():
    print(f"{competition}: {count} matches")

Error for competition_id: 16, season_id: 76. Error message: 'DataFrame' object has no attribute 'competition'
1. Bundesliga: 306 matches
Champions League: 17 matches
Copa del Rey: 3 matches
FA Women's Super League: 326 matches
FIFA U20 World Cup: 1 matches
FIFA World Cup: 147 matches
Indian Super league: 115 matches
La Liga: 868 matches
Liga Profesional: 2 matches
Ligue 1: 377 matches
North American League: 1 matches
NWSL: 36 matches
Premier League: 418 matches
Serie A: 381 matches
UEFA Euro: 51 matches
UEFA Europa League: 3 matches
UEFA Women's Euro: 31 matches
Women's World Cup: 116 matches


In [ ]:
la_liga_id = competitions[competitions['competition_name'] == 'La Liga']['competition_id'].iloc[0]

all_matches = []
seasons = competitions[competitions['competition_id'] == la_liga_id]['season_id'].unique()
for season in seasons:
    matches = sb.matches(la_liga_id, season)
    all_matches.append(matches)

la_liga_matches = pd.concat(all_matches, ignore_index=True)

la_liga_matches['match_date'] = pd.to_datetime(la_liga_matches['match_date'])
la_liga_matches = la_liga_matches.sort_values(by='match_date')
la_liga_matches = la_liga_matches.reset_index(drop=True)

la_liga_matches.head()

,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_status,...,last_updated_360,match_week,competition_stage,stadium,referee,home_managers,away_managers,data_version,shot_fidelity_version,xy_fidelity_version
0,3888713,1974-02-17,21:00:00.000,Spain - La Liga,1973/1974,Real Madrid,Barcelona,0,5,available,...,None,22,Regular Season,Estadio Santiago Bernabéu,NaN,,,1.1.0,2,2
1,68353,2004-10-16,22:00:00.000,Spain - La Liga,2004/2005,Espanyol,Barcelona,0,1,available,...,2021-06-13T16:17:31.694,7,Regular Season,Estadi Olímpic Lluís Companys,Carlos Megía Dávila,Miguel Ángel Lotina Oruecheberría,Frank Rijkaard,1.1.0,2,2
2,68313,2004-10-24,21:00:00.000,Spain - La Liga,2004/2005,Barcelona,Osasuna,3,0,available,...,2021-06-13T16:17:31.694,8,Regular Season,Spotify Camp Nou,Vicente José Lizondo Cortés,Frank Rijkaard,Javier Aguirre Onaindía,1.1.0,2,2
3,68314,2004-12-04,20:00:00.000,Spain - La Liga,2004/2005,Barcelona,Málaga,4,0,available,...,2021-06-13T16:17:31.694,14,Regular Season,Spotify Camp Nou,José Omar Losantos,Frank Rijkaard,Gregorio Manzano Ballesteros,1.1.0,2,2
4,69153,2004-12-11,20:00:00.000,Spain - La Liga,2004/2005,Albacete,Barcelona,1,2,available,...,2021-06-13T16:17:31.694,15,Regular Season,Estadio Carlos Belmonte,Alfonso Perez Burrull,José Manuel González López,Frank Rijkaard,1.1.0,2,2


In [ ]:
la_liga_matches = la_liga_matches.drop_duplicates()

In [ ]:
la_liga_matches.season.unique()

array(['1973/1974', '2004/2005', '2005/2006', '2006/2007', '2007/2008',
       '2008/2009', '2009/2010', '2010/2011', '2011/2012', '2012/2013',
       '2013/2014', '2014/2015', '2015/2016', '2016/2017', '2017/2018',
       '2018/2019', '2019/2020', '2020/2021'], dtype=object)

In [ ]:
# Filter the matches where home_score is equal to away_score
tie_matches = la_liga_matches[la_liga_matches['home_score'] == la_liga_matches['away_score']]

# Count the number of tie matches
num_tie_matches = len(tie_matches)
print(f"Number of tie matches: {num_tie_matches}")


Number of tie matches: 174


In [ ]:
season_2019_2020_matches = la_liga_matches[la_liga_matches['season'] == '2019/2020']

# Count the number of matches in the 2020/2021 season
num_matches = len(season_2019_2020_matches)
print(f"Number of matches in the 2019/2020 season: {num_matches}")

Number of matches in the 2019/2020 season: 33


In [ ]:
season_2020_2021_matches = la_liga_matches[la_liga_matches['season'] == '2020/2021']

# Count the number of matches in the 2020/2021 season
num_matches = len(season_2020_2021_matches)
print(f"Number of matches in the 2020/2021 season: {num_matches}")

Number of matches in the 2020/2021 season: 35


In [ ]:
season_2020_2021_matches = la_liga_matches[la_liga_matches['season'] == '2020/2021']

# Count the number of matches in the 2020/2021 season
num_matches = len(season_2020_2021_matches)
print(f"Number of matches in the 2020/2021 season: {num_matches}")

In [ ]:
len(la_liga_matches)

868

# Graph properties dataframe

In [ ]:
# Function to calculate weighted transitivity
def calculate_weighted_transitivity(G):
      triangles = sum(nx.triangles(G).values()) / 3
      if triangles == 0:
          return 0
      total_weight = 0
      for u, v, d in G.edges(data=True):
          for w in G.neighbors(v):
              if G.has_edge(u, w) and u != w:
                  triangle_weight = min(G[u][v]['weight'], G[v][w]['weight'], G[u][w]['weight'])
                  total_weight += triangle_weight
      return total_weight / triangles if triangles > 0 else 0


In [ ]:
import networkx as nx
import pandas as pd

def graph_theory_match(idmatch):
    teams = la_liga_matches[la_liga_matches.match_id == idmatch]
    home_team = teams.iloc[0]['home_team']
    away_team = teams.iloc[0]['away_team']
    if teams.home_score.iloc[0] > teams.away_score.iloc[0]:
      outcome = 'home_team_winner'
    elif teams.home_score.iloc[0] < teams.away_score.iloc[0]:
      outcome = 'away_team_winner'
    else:
      outcome = 'tie'

    soccer_match = sb.events(match_id=idmatch)
    match_filtered = soccer_match[pd.notna(soccer_match['pass_end_location'])].copy()
    match_filtered[['x', 'y']] = pd.DataFrame(match_filtered.location.tolist(), index=match_filtered.index)

    group = match_filtered.groupby(['player', 'team'])
    players_mean_position = group[['x', 'y']].mean()

    results = {}

    for team in [home_team, away_team]:
        G = nx.Graph()
        match_team = match_filtered[match_filtered.team == team]

        for (player, team_name), position in players_mean_position.iterrows():
            if team_name == team:
                G.add_node(player, pos=(position['x'], position['y']))

        edges = match_team.groupby(['player', 'pass_recipient']).size()
        G.add_weighted_edges_from((player, recipient, weight) for (player, recipient), weight in edges.items())

        total_weight = sum(weight for _, _, weight in G.edges.data('weight', default=1))
        n = G.number_of_nodes()
        weighted_density = total_weight / ((n * (n - 1)) / 2) if n > 1 else 0 # Agregue el sobre dos porque al ser un undirected graph, si no estamos contando dos veces

        weighted_transitivity = calculate_weighted_transitivity(G)
        average_clustering_coefficient = nx.average_clustering(G, weight='weight')

        results[team] = {
            "matchid": la_liga_matches.match_id,
            "density": weighted_density,
            "diameter": nx.diameter(G) if nx.is_connected(G) else None,
            "transitivity": weighted_transitivity,
            "average_clustering": average_clustering_coefficient
        }

    return (match_id,
        home_team, away_team,
        results[home_team]["density"], results[home_team]["diameter"], results[home_team]["transitivity"], results[home_team]["average_clustering"],
        results[away_team]["density"], results[away_team]["diameter"], results[away_team]["transitivity"], results[away_team]["average_clustering"],
        teams.home_score.iloc[0],
        teams.away_score.iloc[0],
        outcome
    )

results = []
for match_id in la_liga_matches['match_id'].unique():
    results.append(graph_theory_match(match_id))

# Define the new columns based on home and away teams
columns = [
    "match_id",
    "home_team", "away_team",
    "home_density", "home_diameter", "home_transitivity", "home_average_clustering",
    "away_density", "away_diameter", "away_transitivity", "away_average_clustering",
    "home_score", "away_score", "outcome"
]

# Convert results to a DataFrame
df = pd.DataFrame(results, columns=columns)
df


,match_id,home_team,away_team,home_density,home_diameter,home_transitivity,home_average_clustering,away_density,away_diameter,away_transitivity,away_average_clustering,home_score,away_score,outcome
0,3888713,Real Madrid,Barcelona,3.500000,2.0,6.361446,0.241817,2.848485,2.0,5.108108,0.235510,0,5,away_team_winner
1,68353,Espanyol,Barcelona,1.417582,2.0,3.366667,0.288377,2.923077,2.0,5.703297,0.188117,0,1,away_team_winner
2,68313,Barcelona,Osasuna,2.472527,2.0,4.238532,0.157029,1.593407,2.0,3.582353,0.261614,3,0,home_team_winner
3,68314,Barcelona,Málaga,4.102564,2.0,6.906250,0.126724,1.461538,2.0,3.992126,0.231941,4,0,home_team_winner
4,69153,Albacete,Barcelona,1.901099,3.0,4.270588,0.121966,2.164835,3.0,5.376000,0.152249,1,2,away_team_winner
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
863,3773586,Barcelona,Granada,5.274725,2.0,8.550000,0.127416,1.380952,3.0,4.104167,0.261264,1,2,away_team_winner
864,3773695,Valencia,Barcelona,2.142857,2.0,4.151515,0.218772,5.384615,2.0,7.545455,0.147729,2,3,away_team_winner
865,3773372,Barcelona,Atlético Madrid,3.171429,2.0,6.142857,0.162664,2.538462,2.0,4.294737,0.124050,0,0,tie
866,3773387,Levante UD,Barcelona,1.950000,3.0,4.775785,0.201404,2.866667,2.0,5.241636,0.120029,3,3,tie


In [ ]:
df.to_csv('df_graph_properties.csv', index=False)

# Historical graph properties dataframe

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

file_path = '/content/drive/Shareddrives/Ceci y Vale/RCEL 506/Código/Archivos csv/df_graph_properties.csv'
df = pd.read_csv(file_path)
df.head()

,match_id,home_team,away_team,home_density,home_diameter,home_transitivity,home_average_clustering,away_density,away_diameter,away_transitivity,away_average_clustering,home_score,away_score,outcome
0,3888713,Real Madrid,Barcelona,3.500000,2.0,6.361446,0.241817,2.848485,2.0,5.108108,0.235510,0,5,away_team_winner
1,68353,Espanyol,Barcelona,1.417582,2.0,3.366667,0.288377,2.923077,2.0,5.703297,0.188117,0,1,away_team_winner
2,68313,Barcelona,Osasuna,2.472527,2.0,4.238532,0.157029,1.593407,2.0,3.582353,0.261614,3,0,home_team_winner
3,68314,Barcelona,Málaga,4.102564,2.0,6.906250,0.126724,1.461538,2.0,3.992126,0.231941,4,0,home_team_winner
4,69153,Albacete,Barcelona,1.901099,3.0,4.270588,0.121966,2.164835,3.0,5.376000,0.152249,1,2,away_team_winner


In [ ]:
n = 3

# Function to create shifted columns for a dataframe
def create_shifted_columns_for_team(df, team_col, n, prefix):
    team_dfs = []
    for team in df[team_col].unique():
        team_df = df[df[team_col] == team].copy()
        for col in ['density', 'transitivity', 'average_clustering', 'diameter']:
            for i in range(1, n + 1):
                shifted_col_name = f"{prefix}_{col}_{i}"
                team_df[shifted_col_name] = team_df[f"{prefix}_{col}"].shift(i)
        team_dfs.append(team_df)
    return pd.concat(team_dfs)

# Apply the function to create shifted columns for home and away teams
home_df_shifted = create_shifted_columns_for_team(df, 'home_team', n, 'home')
away_df_shifted = create_shifted_columns_for_team(df, 'away_team', n, 'away')

# Select only the shifted columns along with match_id for merging
home_cols_shifted = ['match_id'] + [col for col in home_df_shifted.columns if 'home_' in col and col not in df.columns]
away_cols_shifted = ['match_id'] + [col for col in away_df_shifted.columns if 'away_' in col and col not in df.columns]

# Merge the original dataframe with the shifted data for the home team
merged_home_df = pd.merge(df, home_df_shifted[home_cols_shifted], on="match_id") # on is used to merge based on match_id

# Merge the result with the shifted data for the away team
final_merged_df = pd.merge(merged_home_df, away_df_shifted[away_cols_shifted], on="match_id")

# Define the desired columns in the correct order
columns = [
    'match_id', 'home_team', 'away_team', 'outcome',
    'home_density', 'home_diameter', 'home_transitivity', 'home_average_clustering',
    'away_density', 'away_diameter', 'away_transitivity', 'away_average_clustering',
    'home_score', 'away_score'
] + home_cols_shifted[1:] + away_cols_shifted[1:]  # Exclude 'match_id' from shifted columns

final_df = final_merged_df[columns]
final_df.head()

,match_id,home_team,away_team,outcome,home_density,home_diameter,home_transitivity,home_average_clustering,away_density,away_diameter,...,away_density_3,away_transitivity_1,away_transitivity_2,away_transitivity_3,away_average_clustering_1,away_average_clustering_2,away_average_clustering_3,away_diameter_1,away_diameter_2,away_diameter_3
0,3888713,Real Madrid,Barcelona,away_team_winner,3.500000,2.0,6.361446,0.241817,2.848485,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,68353,Espanyol,Barcelona,away_team_winner,1.417582,2.0,3.366667,0.288377,2.923077,2.0,...,NaN,5.108108,NaN,NaN,0.235510,NaN,NaN,2.0,NaN,NaN
2,68313,Barcelona,Osasuna,home_team_winner,2.472527,2.0,4.238532,0.157029,1.593407,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,68314,Barcelona,Málaga,home_team_winner,4.102564,2.0,6.906250,0.126724,1.461538,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,69153,Albacete,Barcelona,away_team_winner,1.901099,3.0,4.270588,0.121966,2.164835,3.0,...,NaN,5.703297,5.108108,NaN,0.188117,0.23551,NaN,2.0,2.0,NaN


In [ ]:
# Checking if it is ok
selected_columns = final_df.loc[final_df['home_team'] == 'Real Madrid', ['home_team', 'away_team', 'home_density', 'home_density_1', 'home_density_2', 'home_density_3']]
selected_columns.head()

,home_team,away_team,home_density,home_density_1,home_density_2,home_density_3
0,Real Madrid,Barcelona,3.500000,NaN,NaN,NaN
12,Real Madrid,Barcelona,3.269231,3.500000,NaN,NaN
31,Real Madrid,Barcelona,2.282051,3.269231,3.500000,NaN
76,Real Madrid,Barcelona,2.901099,2.282051,3.269231,3.500000
108,Real Madrid,Barcelona,2.252747,2.901099,2.282051,3.269231


In [ ]:
df.head()

,match_id,home_team,away_team,home_density,home_diameter,home_transitivity,home_average_clustering,away_density,away_diameter,away_transitivity,away_average_clustering,home_score,away_score,outcome
0,3888713,Real Madrid,Barcelona,3.500000,2.0,6.361446,0.241817,2.848485,2.0,5.108108,0.235510,0,5,away_team_winner
1,68353,Espanyol,Barcelona,1.417582,2.0,3.366667,0.288377,2.923077,2.0,5.703297,0.188117,0,1,away_team_winner
2,68313,Barcelona,Osasuna,2.472527,2.0,4.238532,0.157029,1.593407,2.0,3.582353,0.261614,3,0,home_team_winner
3,68314,Barcelona,Málaga,4.102564,2.0,6.906250,0.126724,1.461538,2.0,3.992126,0.231941,4,0,home_team_winner
4,69153,Albacete,Barcelona,1.901099,3.0,4.270588,0.121966,2.164835,3.0,5.376000,0.152249,1,2,away_team_winner


In [ ]:
average_density = (df['home_density'].mean() + df['away_density'].mean()) / 2
average_diameter = (df['home_diameter'].mean() + df['away_diameter'].mean()) / 2
average_transitivity = (df['home_transitivity'].mean() + df['away_transitivity'].mean()) / 2
average_clustering = (df['home_average_clustering'].mean() + df['away_average_clustering'].mean()) / 2

print('Average Density: ' + str(average_density))
print('Average Diameter: ' + str(average_diameter))
print('Average Transitivity: ' + str(average_transitivity))
print('Average Clustering: ' + str(average_clustering))

Average Density: 2.8858550963245664
Average Diameter: 2.080439814814815
Average Transitivity: 5.507164552799928
Average Clustering: 0.18039354107364924


In [ ]:
# I will replace the NaN values with the average values
average_density = (df['home_density'].mean() + df['away_density'].mean()) / 2
average_diameter = (df['home_diameter'].mean() + df['away_diameter'].mean()) / 2
average_transitivity = (df['home_transitivity'].mean() + df['away_transitivity'].mean()) / 2
average_clustering = (df['home_average_clustering'].mean() + df['away_average_clustering'].mean()) / 2

for i in range(1, n + 1):
    final_df[f'home_density_{i}'].fillna(average_density, inplace=True) #inplace true means that I am modifying the df, not creating a new one
    final_df[f'away_density_{i}'].fillna(average_density, inplace=True)
    final_df[f'home_diameter_{i}'].fillna(average_diameter, inplace=True)
    final_df[f'away_diameter_{i}'].fillna(average_diameter, inplace=True)
    final_df[f'home_transitivity_{i}'].fillna(average_transitivity, inplace=True)
    final_df[f'away_transitivity_{i}'].fillna(average_transitivity, inplace=True)
    final_df[f'home_average_clustering_{i}'].fillna(average_clustering, inplace=True)
    final_df[f'away_average_clustering_{i}'].fillna(average_clustering, inplace=True)

final_df.head()

<ipython-input-81-692a50b6bff4>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df[f'home_density_{i}'].fillna(average_density, inplace=True) #inplace true means that I am modifying the df, not creating a new one
<ipython-input-81-692a50b6bff4>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df[f'away_density_{i}'].fillna(average_density, inplace=True)
<ipython-input-81-692a50b6bff4>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-vers

,match_id,home_team,away_team,outcome,home_density,home_diameter,home_transitivity,home_average_clustering,away_density,away_diameter,...,away_density_3,away_transitivity_1,away_transitivity_2,away_transitivity_3,away_average_clustering_1,away_average_clustering_2,away_average_clustering_3,away_diameter_1,away_diameter_2,away_diameter_3
0,3888713,Real Madrid,Barcelona,away_team_winner,3.500000,2.0,6.361446,0.241817,2.848485,2.0,...,2.885855,5.507165,5.507165,5.507165,0.180394,0.180394,0.180394,2.08044,2.08044,2.08044
1,68353,Espanyol,Barcelona,away_team_winner,1.417582,2.0,3.366667,0.288377,2.923077,2.0,...,2.885855,5.108108,5.507165,5.507165,0.235510,0.180394,0.180394,2.00000,2.08044,2.08044
2,68313,Barcelona,Osasuna,home_team_winner,2.472527,2.0,4.238532,0.157029,1.593407,2.0,...,2.885855,5.507165,5.507165,5.507165,0.180394,0.180394,0.180394,2.08044,2.08044,2.08044
3,68314,Barcelona,Málaga,home_team_winner,4.102564,2.0,6.906250,0.126724,1.461538,2.0,...,2.885855,5.507165,5.507165,5.507165,0.180394,0.180394,0.180394,2.08044,2.08044,2.08044
4,69153,Albacete,Barcelona,away_team_winner,1.901099,3.0,4.270588,0.121966,2.164835,3.0,...,2.885855,5.703297,5.108108,5.507165,0.188117,0.235510,0.180394,2.00000,2.00000,2.08044


In [ ]:
# Checking if it is ok
selected_columns = final_df.loc[final_df['home_team'] == 'Real Madrid', ['home_team', 'away_team', 'home_density', 'home_density_1', 'home_density_2', 'home_density_3']]
selected_columns.head()

,home_team,away_team,home_density,home_density_1,home_density_2,home_density_3
0,Real Madrid,Barcelona,3.500000,2.885855,2.885855,2.885855
12,Real Madrid,Barcelona,3.269231,3.500000,2.885855,2.885855
31,Real Madrid,Barcelona,2.282051,3.269231,3.500000,2.885855
76,Real Madrid,Barcelona,2.901099,2.282051,3.269231,3.500000
108,Real Madrid,Barcelona,2.252747,2.901099,2.282051,3.269231


In [ ]:
final_df.to_csv('df_historical_graph_properties.csv', index=False)